In [ ]:
#Link para Colaboratory
#https://colab.research.google.com/drive/1wZxhtAX_EXF5O4_I3vXTSG0BaZHlbeTG?usp=sharing

# imports necessários:
import pandas as pd
from pathlib import PurePath, PurePosixPath, PosixPath 
from datetime import datetime
import sqlite3 as sql


## Coleta de dados: site IBGE

In [ ]:
#Coletar os dados dos municípios de Minas Gerais
url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#MG'

#Inserir os dados no Data Frame ibge_dados_mg
ibge_dados_mg = pd.DataFrame(pd.read_html(url, match='Municípios de Minas Gerais')[0])

#Ajustar a nomenclatura das colunas do Data Frame:
ibge_dados_mg = ibge_dados_mg.rename(columns = {'Municípios de Minas Gerais': 'nomeMunic', 'Códigos': 'codMunic'})

#Adicionar coluna ao Data Frame:
ibge_dados_mg['dtCarga'] = datetime.today().strftime('%d/%m/%Y %H:%M')

#Ajustar index do Data Frame
ibge_dados_mg.index.name = 'idMunic'
ibge_dados_mg.index = ibge_dados_mg.index + 1


## Configuração Banco de Dados: ODS e DW - SQLite

In [ ]:
##Indicar local dos arquivos:
endereco = PosixPath('/')

#Indicar endereço das Bases de dados
bd_ods = endereco / "ibgeMgODS.db"
bd_dw = endereco / "ibgeMgDW.db"

#Verificar a existência das bases de dados
if endereco.exists():
    if (bd_ods.exists() and bd_dw.exists()):
        print('Bancos de dados já existem!')
    else:
        bd_ods.touch()
        bd_dw.touch()
        print('Bancos de dados criados!')
else:
    print('Endereço não existe, favor verificar!')
    

Bancos de dados criados!


## Inserir informações - ODS

In [ ]:
#Conectar no ODS
conexao_ods = sql.connect(bd_ods)

#Criar e/ou popular tabela tbLogMunic com o DF ibge_dados_mg
ibge_dados_mg.to_sql('tbLogMunic', conexao_ods, if_exists='append')

conexao_ods.commit()
conexao_ods.close()

print('Carga ODS concluída')

Carga ODS concluída


## Inserir informações - DW

In [ ]:
#Conectar no DW
conexao_dw = sql.connect(bd_dw)

#Criar Data Frame para popular o DW
ibge_dados_mg_dw = ibge_dados_mg[['codMunic','nomeMunic']]

#Limpar dados anteriores e inserir novas informações da dimensão dMunicipio com o Data Frame ibge_dados_mg_dw
ibge_dados_mg_dw.to_sql('dMunicipio', conexao_dw, if_exists='replace')

#Confirmar ações e encerrar conexão
conexao_dw.commit()
conexao_dw.close()

print('Carga DW concluída')

Carga DW concluída


## Visualizar informnações DW


In [ ]:
from pickle import NONE
#Conectar no DW
conexao_dw = sql.connect(bd_dw)

#Consulta a ser retornada
consulta = '''
    SELECT
        nomeMunic [Nome do Município]
    FROM dMunicipio
'''
exibir = pd.read_sql(consulta,conexao_dw)

#Confirmar ações e encerrar conexão
conexao_dw.commit()
conexao_dw.close()

#Exibir informações do DW
exibir.index.name = ''
exibir.index = exibir.index + 1
exibir


,Nome do Município
,
1,Abadia dos Dourados
2,Abaeté
3,Abre Campo
4,Acaiaca
5,Açucena
...,...
849,Virginópolis
850,Virgolândia
851,Visconde do Rio Branco
